In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd
import tensorflow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("../data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition', 
                        'koi_period', 
                        'koi_impact', 
                        'koi_duration', 
                        'koi_depth',
                        'koi_prad',
                        'koi_teq',
                        'koi_slogg'
                       ]]

selected_features

,koi_disposition,koi_period,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_slogg
0,CONFIRMED,54.418383,0.586,4.50700,874.8,2.83,443,4.467
1,FALSE POSITIVE,19.899140,0.969,1.78220,10829.0,14.60,638,4.544
2,FALSE POSITIVE,1.736952,1.276,2.40641,8079.2,33.46,1395,4.564
3,CONFIRMED,2.525592,0.701,1.65450,603.3,2.75,1406,4.438
4,CONFIRMED,4.134435,0.762,3.14020,686.0,2.77,1160,4.486
...,...,...,...,...,...,...,...,...
6986,FALSE POSITIVE,8.589871,0.765,4.80600,87.7,1.11,929,4.296
6987,FALSE POSITIVE,0.527699,1.252,3.22210,1579.2,29.35,2088,4.529
6988,CANDIDATE,1.739849,0.043,3.11400,48.5,0.72,1608,4.444
6989,FALSE POSITIVE,0.681402,0.147,0.86500,103.6,1.07,2218,4.447


# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [7]:
# Assign X (data) and y (target)

X = selected_features.drop(columns="koi_disposition")
y = selected_features["koi_disposition"].values.reshape(-1,1)

print(X.shape, y.shape)

(6991, 7) (6991, 1)


In [8]:
# Split the data into training and testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
X_train.head()

,koi_period,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_slogg
6122,6.768901,0.150,3.61600,123.1,1.24,1017,4.327
6370,0.733726,0.291,2.30900,114.6,0.86,1867,4.578
2879,7.652707,0.970,79.89690,641.1,3.21,989,4.481
107,7.953547,0.300,2.63120,875.4,2.25,696,4.536
29,4.959319,0.831,2.22739,9802.0,12.21,1103,4.359


In [6]:
# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data

# Train the Model



In [ ]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)